In [1]:
import pandas as pd
import awswrangler as wr
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import image_uris 
from sagemaker.session import s3_input, Session
import urllib
import os
import numpy as np
from sagemaker.predictor import csv_serializer
from time import gmtime, strftime

In [2]:
import credentials as cr

In [3]:
import importlib
importlib.reload(cr)

<module 'credentials' from '/Users/gaby/Documents/GitHub/sm/credentials.py'>

In [4]:
bucket_name = 'aws-sm-bucket' 
my_region = boto3.session.Session().region_name 
print(my_region)

us-east-2


In [5]:
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://aws-sm-bucket/xgboost-as-a-built-in-algo/output


## Read local data to S3

In [6]:
name_data = 'test_data'
#boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, f'{name}/{name}.csv')).upload_file(f'{name}.csv')


## Deploy already trained model to endpoint

__Choose the model__

In [7]:
bucket = boto3.resource('s3').Bucket(bucket_name)

In [74]:
models =[ obj.key for obj in bucket.objects.all() if 'model' in obj.key]
models

['xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-03-18-09-57-40-076/output/model.tar.gz',
 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-03-18-10-26-05-864/output/model.tar.gz',
 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-03-18-10-29-00-410/output/model.tar.gz']

In [75]:
model_path = models[0]
model_path

'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-03-18-09-57-40-076/output/model.tar.gz'

In [91]:
import re
model_name = re.split( '\/', model_path)[-1]
model_name = re.split( '\.', model_name)[0]
model_name = model_name + '-v2'
model_name

'model-v2'

__Create sagemaker client__

In [94]:
model_url = f's3://{bucket_name}/{model_path}'
sm_client = boto3.client('sagemaker')
print (model_url)

s3://aws-sm-bucket/xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-03-18-09-57-40-076/output/model.tar.gz


__Create container with model__

In [95]:
container = image_uris.retrieve(region = boto3.Session().region_name, framework='xgboost', version='1.0-1')
primary_container = { 'Image': container, 'ModelDataUrl': model_url,}


create_model_response2 = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = cr.role,
    PrimaryContainer = primary_container)

print(create_model_response2['ModelArn'])

arn:aws:sagemaker:us-east-2:342342686540:model/model-v2


__Configure the endpoint__

In [96]:
instance_type='ml.t2.medium'

In [97]:
from time import gmtime, strftime

endpoint_config_name = 'XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':instance_type,
        'InitialInstanceCount':1,
        'InitialVariantWeight':1,
        'ModelName': model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])


XGBoostEndpointConfig-2021-03-18-12-27-24
Endpoint Config Arn: arn:aws:sagemaker:us-east-2:342342686540:endpoint-config/xgboostendpointconfig-2021-03-18-12-27-24


__Create the endpoint__

In [98]:
%%time
## creating the endpoint
import time

endpoint_name = 'XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(EndpointName = endpoint_name, 
                                                     EndpointConfigName = endpoint_config_name)

print(create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName = endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

XGBoostEndpoint-2021-03-18-12-28-17
arn:aws:sagemaker:us-east-2:342342686540:endpoint/xgboostendpoint-2021-03-18-12-28-17
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:342342686540:endpoint/xgboostendpoint-2021-03-18-12-28-17
Status: InService
CPU times: user 252 ms, sys: 46.3 ms, total: 298 ms
Wall time: 9min 55s


__Check list of endpoints__

In [99]:
endpoints = sm_client.list_endpoints(SortBy='Status')['Endpoints']
endpoints

[{'EndpointName': 'XGBoostEndpoint-2021-03-18-12-28-17',
  'EndpointArn': 'arn:aws:sagemaker:us-east-2:342342686540:endpoint/xgboostendpoint-2021-03-18-12-28-17',
  'CreationTime': datetime.datetime(2021, 3, 18, 13, 28, 23, 602000, tzinfo=tzlocal()),
  'LastModifiedTime': datetime.datetime(2021, 3, 18, 13, 37, 35, 768000, tzinfo=tzlocal()),
  'EndpointStatus': 'InService'}]

## Test container

In [100]:
test_data = pd.read_csv('test_data.csv')
runtime_client = boto3.client('sagemaker-runtime')
payload = test_data.copy()
payload = payload.drop(payload.columns[0], axis=1)#.values 
payload.to_csv('payload_data.csv', index=False, header=False)


In [101]:
with open('payload_data.csv', 'r') as f:
    payload = f.read().strip()

response = runtime_client.invoke_endpoint(
                            EndpointName = endpoint_name, 
                            ContentType='text/csv', 
                            Body= payload)
result = response['Body'].read()

In [102]:


predictions_array = np.fromstring(result, sep=',') # and turn the prediction into an array
cm = pd.crosstab(index=test_data[test_data.columns[0]], 
                 columns=np.round(predictions_array), 
                 rownames=['Observed'], 
                 colnames=['Predicted'])
print(cm)
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

Predicted    0.0  1.0
Observed             
0          10783  152
1           1135  286

Overall Classification Rate: 89.6%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10783)    35% (152)
Purchase        10% (1135)     65% (286) 



## Delete models

In [116]:
models = sm_client.list_models()
#https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/models
models = [x['ModelName'] for x in models['Models']]
models

['model-v2']

In [117]:
for x in models:
    print(x)
    if x != model_name:
        print('****** delete: ' + str(x))
        sm_client.delete_model(ModelName = str(x))


model-v2


## Delete Endpoints

In [109]:
#if you want to delete a list of endpoints keeping the last "keep_ast" ones.
keep_last = 0
endpoints = sm_client.list_endpoints(SortBy='Status')['Endpoints']
for i in range(keep_last, len(endpoints) - keep_last):# exclude last training and delete
    name = endpoints[i]['EndpointName']
    print(name)
    sagemaker.Session().delete_endpoint(name)

XGBoostEndpoint-2021-03-18-12-28-17


## Delete S3 objects

In [ ]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
print(bucket_to_delete.objects.all())
bucket_to_delete.objects.all().delete()